# Road Traffic Sign Classify Model

The work below attempt to create/evaluate machine learning models and to train a model to classify images of European road traffic signs. [here]().

* There are roughly 2000 instances and 20 features/attributes for the training data.
* There are several models with different performances considerated in this work.
* More details about the analysis/evaluation of the dataset and the models can be found in this [analysis report](https://newborn-life-expectancy-model.notebook.us-east-1.sagemaker.aws/lab/tree/newborn-life-expectancy-model/Report.docx).

>Author: ***Daniel Dao***

<hr size="5" />